<a href="https://colab.research.google.com/github/natashacarpcast/nlp-finalproject/blob/main/top2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [1]:
pip install top2vec

In [151]:
from google.colab import drive
import pandas as pd
from top2vec import Top2Vec
import nltk
import numpy as np
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Upload data

In [3]:
filepath = "/content/drive/My Drive/UChicago/nlp/cleaned_submissions.csv"
data = pd.read_csv(filepath)

<ipython-input-3-215e78c66f39>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filepath)


In [68]:
data.shape

(561943, 20)

#Top2Vec Topic modeling

Convert documents into a list of strings

In [69]:
docs = data["title+bodytext"].to_list()
docs[0:5]

['tips for being successful in the workplace as mentioned from someone making gt 100kyr ',
 'if you are using google chrome start typing the web address of the site you want to search in the address bar if google chrome has a record of the sites search engine it automatically offers you the option to search that site ',
 'when applying eyedrops dont drop them inside the middle of the eye go for the inside corner ',
 'beer before liquor  never been sicker liquor before beer  always in the clear ',
 'when your significant other isnt around take measurements of everything heshe wears and write them down ']

Get docs_ids to map with documents in the model

In [72]:
docs_ids = data["id"].to_list()
docs_ids[0:5]

['fs5kt', 'fs4yb', 'fs4xb', 'fros1', 'fromj']

Create custom tokenizer to do stop-word cleaning specific to the model, so it doesn't retrieve words as "lpt" or "lpt request

In [73]:
custom_stopwords = ['lpt', 'request', 'lpts', 'lptr', 'lptgood', 'lptnext',
                    'lptdo', 'lptrequest', 'lp', 'lptyou', "pro", "tip",
                    "advice", "tips", "helpful", "idea", "advices", "protip",
                    "protips"]

def remove_stopwords(text):
    text = [word for word in text if word not in custom_stopwords]
    return text

def tok(text):
    import gensim
    text = gensim.utils.simple_preprocess(text)
    text = remove_stopwords(text)
    return text

Top2Vec model

- Use topic_merge_data of 0.3 to decrease number of topics (initial run yielded 1857 topics)
- Use ngrams for capturing interesting ngrams
- Use custom-made tokenizer
- The all-MiniLM-L6-v2 model is particularly suitable for this task for several reasons. First, it was trained on Reddit data, which allows it to better capture the idiosyncrasies of the corpus. Second, it is specifically recommended for clustering tasks, making it a strong choice for this type of analysis.
- Leverage GPU for dimensionality reduction and clustering
- Map documents ids to posts

In [81]:
model = Top2Vec(docs, topic_merge_delta=0.2, ngram_vocab=True,
                tokenizer= tok, embedding_model='all-MiniLM-L6-v2',
                gpu_umap=True, gpu_hdbscan=True, document_ids=docs_ids)

2025-02-28 00:45:29,402 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2025-02-28 00:46:20,744 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model
INFO:top2vec:Downloading all-MiniLM-L6-v2 model
2025-02-28 00:46:21,598 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2025-02-28 00:49:19,979 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2025-02-28 00:49:45,101 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents


[I] [00:49:28.414459] Transform can only be run with brute force. Using brute force.


2025-02-28 00:50:06,998 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [82]:
model.get_num_topics()

1413

In [83]:
topic_sizes, topic_nums = model.get_topic_sizes()

In [84]:
topic_sizes

array([5340, 4674, 4227, ...,   31,   26,   25])

Look at top 20 topics

In [85]:
topic_words, word_scores, topic_nums = model.get_topics(20)

In [86]:
topic_words

array([['hygiene routine', 'alcohol wipes', 'personal hygiene',
        'lysol wipes', 'wet wipes', 'moist wipes', 'toilette paper',
        'toilet paper', 'sanitary pads', 'toilet clogs',
        'hygiene products', 'eliminate odors', 'clog toilet',
        'breath stinks', 'antibacterial wipes', 'wet washcloth',
        'sanitizing wipes', 'wet napkin', 'while showering',
        'sanitizing wipe', 'wiper fluid', 'dry cleaners',
        'stinky breath', 'unclog toilet', 'deodorants air',
        'soapbody wash', 'baby wipes', 'washing detergent',
        'clorox wipes', 'hand soaps', 'cleaning wipes', 'soapy water',
        'wash cloths', 'bad odors', 'hand soap', 'wash cloth',
        'apply deodorant', 'wet rag', 'breath smells', 'clorox wipe',
        'taking bath', 'flushable wet', 'liquid detergent',
        'public restroom', 'deodorant streaks', 'disinfectant wipe',
        'dry cleaned', 'laundry detergent', 'smelly sponges',
        'antiperspirant deodorant'],
       ['pro

Define labels - Exclude a couple as they are not informative enough

0. **Hygiene and Cleaning**  
1. **Savings**  
2. **Telecommunications**  
3. **Eating Behaviour**  
4. *(Exclude)*
5. **Sleep**  
6. **Traffic and Road Safety**  
7. **Restaurant and Food Ordering**  
8. **Hair care and shaving**  
9. **Asking and offering help**  
10. *(Exclude)*  
11. **Communication and Conversations**  
12. **Pets**  
13. *(Exclude)*
14. **Money management**  
15. **Self-Improvement and Well-being**  
16. **Friendships and Social Relationships**  
17. **Debating and Argumenting**  
18. **Career and Education**  
19. **Email and Spam Management**  


Save for future use if needed

In [87]:
model.save("/content/drive/My Drive/UChicago/nlp/model_top2vec")

# Classify documents by topic

In [93]:
topic_nums, topic_score, topics_words, word_scores = model.get_documents_topics(doc_ids=docs_ids)

In [153]:
data["topic"] = topic_nums

Filter the dataset for the top 20 topics

In [154]:
data_top20_topics = data[data.topic < 20]
data_top20_topics.shape


(64030, 21)

Remove weird topics

In [155]:
data_top20_topics = data_top20_topics[data_top20_topics.topic != 4]
data_top20_topics = data_top20_topics[data_top20_topics.topic != 13]
data_top20_topics = data_top20_topics[data_top20_topics.topic != 10]

In [156]:
data_top20_topics.shape

(54568, 21)

#Manually check documents assignations and sort by reddit popularity (score)

In [157]:
data_top20_topics.columns

#Change name because we realized the column only has the title
data_top20_topics = data_top20_topics.rename(columns={'title+bodytext': 'title'})

In [158]:
def show_top10(data, topic_n):
  with pd.option_context('display.max_colwidth', None):
    display(data_top20_topics[data_top20_topics.topic == topic_n].sort_values(by="score",
                  ascending=False)[["id", "topic", "title","link","score"]][0:10])


In [159]:
labels = {
    0: "Hygiene and Cleaning",
    1: "Savings",
    2: "Telecommunications",
    3: "Eating Behaviour",
    5: "Sleep",
    6: "Traffic and Road Safety",
    7: "Restaurant and Food Ordering",
    8: "Hair care and shaving",
    9: "Asking and offering help",
    11: "Communication and Conversations",
    12: "Pets",
    14: "Money management",
    15: "Self-Improvement and Well-being",
    16: "Friendships and Social Relationships",
    17: "Debating and Argumenting",
    18: "Career and Education",
    19: "Email and Spam Management"
}

for topic_num in range(20):
  if topic_num != 4 and topic_num != 13 and topic_num != 10:
    print(f"Topic: {labels.get(topic_num)}")
    display(show_top10(data_top20_topics, topic_num))



Topic: Hygiene and Cleaning


,id,topic,title,link,score
357293,e0eqyz,0,lpt tips what you should never to bring to a dry cleaners,https://www.reddit.com/r/LifeProTips/comments/e0eqyz/lpt_tips_what_you_should_never_to_bring_to_a_dry/,47057
434738,jtwx6p,0,lpt dont let the message of washing your hands get lost in the wear a mask dialogue,https://www.reddit.com/r/LifeProTips/comments/jtwx6p/lpt_dont_let_the_message_of_washing_your_hands/,35519
376860,fevekc,0,lpt to encourage kids to wash their hands get them a special soap,https://www.reddit.com/r/LifeProTips/comments/fevekc/lpt_to_encourage_kids_to_wash_their_hands_get/,32131
336672,ci5114,0,lpt at a restaurant always wash your hands after ordering menus are generally the dirtiest thing you touch in a restaurant,https://www.reddit.com/r/LifeProTips/comments/ci5114/lpt_at_a_restaurant_always_wash_your_hands_after/,31412
244613,7tcvc5,0,lpt wash your hands after ordering in a restaurant the menu is generally the dirtiest thing you can touch,https://www.reddit.com/r/LifeProTips/comments/7tcvc5/lpt_wash_your_hands_after_ordering_in_a/,19909
493209,qwqm0c,0,lpt your towel is a major influence on how you smell after a shower,https://www.reddit.com/r/LifeProTips/comments/qwqm0c/lpt_your_towel_is_a_major_influence_on_how_you/,18156
538136,10ep70l,0,lpt drink water after youre finished your coffee so your breath doesnt stink at work,https://www.reddit.com/r/LifeProTips/comments/10ep70l/lpt_drink_water_after_youre_finished_your_coffee/,14534
526734,xadt8o,0,lpt request how to tell someone they need better hygiene,https://www.reddit.com/r/LifeProTips/comments/xadt8o/lpt_request_how_to_tell_someone_they_need_better/,8031
416067,iffr8o,0,lpt when you find yourself having a harder and harder time matching your socks or are finding too many socks with holes in them get rid of them all and buy a new set the sock reset makes your life better in every single way possible,https://www.reddit.com/r/LifeProTips/comments/iffr8o/lpt_when_you_find_yourself_having_a_harder_and/,7135
513957,uloyk5,0,lpt wash your member before receiving oral even if you think its clean,https://www.reddit.com/r/LifeProTips/comments/uloyk5/lpt_wash_your_member_before_receiving_oral_even/,6634


None

Topic: Savings


,id,topic,title,link,score
421853,itmoxw,1,lpt buying good quality stuff preowned rather than bad quality stuff new makes a lot of sense if youre on a budget,https://www.reddit.com/r/LifeProTips/comments/itmoxw/lpt_buying_good_quality_stuff_preowned_rather/,68888
381710,fq401b,1,lpt if you manage a store put a sign out front listing the current hot items that are out of stock this will prevent unnecessary traffic in your store and reduce potential exposure for your employees and customers miscellaneous,https://www.reddit.com/r/LifeProTips/comments/fq401b/lpt_if_you_manage_a_store_put_a_sign_out_front/,56847
459564,m3xceb,1,lpt if youre on the fence about buying anything ask yourself if i had to choose between being given this item or its equivalent value in money which would i take,https://www.reddit.com/r/LifeProTips/comments/m3xceb/lpt_if_youre_on_the_fence_about_buying_anything/,41247
466361,mxdnvd,1,lpt if you have been eyeing something on etsy make sure to favorite that item the seller will get notified and often directly send you a promodiscount code its a good way to get 1015 off,https://www.reddit.com/r/LifeProTips/comments/mxdnvd/lpt_if_you_have_been_eyeing_something_on_etsy/,38555
205292,6kqaj9,1,lpt when youre thinking about buying something you dont necessarily need imagine the item in one hand and the cash in the other which one would you take,https://www.reddit.com/r/LifeProTips/comments/6kqaj9/lpt_when_youre_thinking_about_buying_something/,35376
426447,j5rg1d,1,lpt be wary of any product which advertises the cost in a per day format this is deliberately done to fool people into thinking they can afford it,https://www.reddit.com/r/LifeProTips/comments/j5rg1d/lpt_be_wary_of_any_product_which_advertises_the/,34791
368470,erix4f,1,lpt pay attention to the end of prices while shopping at costco,https://www.reddit.com/r/LifeProTips/comments/erix4f/lpt_pay_attention_to_the_end_of_prices_while/,33025
321327,bde5xt,1,lpt if you want to give something away online still add a price even if its very low for that item it will scare away 90 of those who will just relist the item immediately plus itll pleasant surprise for someone when you give them it free,https://www.reddit.com/r/LifeProTips/comments/bde5xt/lpt_if_you_want_to_give_something_away_online/,30233
471771,nnmdqd,1,lpt you dont save money buying items that are on sale if you werent planning on buying them for their original price,https://www.reddit.com/r/LifeProTips/comments/nnmdqd/lpt_you_dont_save_money_buying_items_that_are_on/,29586
370635,exryih,1,lpt if you measure every purchase in hours you worked instead of price youd be less of a consumer,https://www.reddit.com/r/LifeProTips/comments/exryih/lpt_if_you_measure_every_purchase_in_hours_you/,27763


...

,id,topic,title,link,score
278840,98d589,18,lpt for those of you going to college for the first time this month go to class no matter how hungover tired or busy you may be being present is the most important factor in succeeding in your first year as you adjust to living independently missing class is a slippery slope to failing out,https://www.reddit.com/r/LifeProTips/comments/98d589/lpt_for_those_of_you_going_to_college_for_the/,92389
252799,85n98q,18,lpt when you know where you are going to college make sure to decline the others people often end up on waitlists and the sooner that you decline the more likely they may be to get in it could be their dreamreach school,https://www.reddit.com/r/LifeProTips/comments/85n98q/lpt_when_you_know_where_you_are_going_to_college/,71765
443047,kg90h7,18,lpt dont go to college if you dont have a good plan of what youre going to do there college isnt neccessarily the next step in life after high school and you will waste time and money going there without a passion,https://www.reddit.com/r/LifeProTips/comments/kg90h7/lpt_dont_go_to_college_if_you_dont_have_a_good/,71497
439112,k67fhx,18,lpt finish your degree even if you ultimately wont go into your studied field future employers may simply pay you more for having a degree,https://www.reddit.com/r/LifeProTips/comments/k67fhx/lpt_finish_your_degree_even_if_you_ultimately/,65243
285110,9idbfg,18,lpt high schoolers dont think you have to go to college to be successful not great at school look at learning a trade you can make a good sustainable income as an electrician plumber welder etc you can enroll in a technical college or better yet find a apprenticeship at a larger company,https://www.reddit.com/r/LifeProTips/comments/9idbfg/lpt_high_schoolers_dont_think_you_have_to_go_to/,63529
432047,jlqdk6,18,lpt if you are in your 20s study personal finance like your life depended on it investing 401 k roth ira rrsp and tfsa in canada credit score credit and debit cards debts loans bankruptcy etc its a travesty that its not mandatory learning in school,https://www.reddit.com/r/LifeProTips/comments/jlqdk6/lpt_if_you_are_in_your_20s_study_personal_finance/,61980
289699,9qpkhd,18,lpt dont attend an expensive private university if you have to pay for it entirely with loans dont aim for state schools or attend community college for 2 years and then try to get into some place affordable but dont take out a mortgage for a bachelors it will hold you back for years,https://www.reddit.com/r/LifeProTips/comments/9qpkhd/lpt_dont_attend_an_expensive_private_university/,59353
204296,6jtd0h,18,lpt if you started college at a 2 year school then transfer to a 4 year school and dropped out without a degree contact the 2 year school about reverse transfer programs you may have already earned an associates degree that you are not getting credit for,https://www.reddit.com/r/LifeProTips/comments/6jtd0h/lpt_if_you_started_college_at_a_2_year_school/,48956
217863,6wjiu0,18,lpt get to know your professors early on in your later years good relationships with professors can lead to recommendations and research and job opportunities,https://www.reddit.com/r/LifeProTips/comments/6wjiu0/lpt_get_to_know_your_professors_early_on_in_your/,48880
496031,rfjz2w,18,lpt dont get too attached to dream schools or dream jobs before you really get some firsthand experience most of these placescareers are considered good because of abstract prestige not based on having healthy environments that promote growth and personal happiness,https://www.reddit.com/r/LifeProTips/comments/rfjz2w/lpt_dont_get_too_attached_to_dream_schools_or/,46569


None

Topic: Email and Spam Management


,id,topic,title,link,score
430204,jgpzpr,19,lpt it only takes about 23 weeks of clicking unsubscribe on every single marketing email you receive to change your inbox and your life forever,https://www.reddit.com/r/LifeProTips/comments/jgpzpr/lpt_it_only_takes_about_23_weeks_of_clicking/,72847
452842,lahtmx,19,lpt if you have to give your email to a company use the companys name as your listed first name that way if they sell your information to third parties the spam emails will be autofilled with the original companys name instead of your own so you know who sold your info,https://www.reddit.com/r/LifeProTips/comments/lahtmx/lpt_if_you_have_to_give_your_email_to_a_company/,56787
419878,iomh79,19,lpt reminder that you can create unlimited email addresses in gmail by adding a plus sign followed by a word to your existing address use different emails such as emailnetflixgmailcom emailspotify gmailcom for different services,https://www.reddit.com/r/LifeProTips/comments/iomh79/lpt_reminder_that_you_can_create_unlimited_email/,47217
155433,5hexgl,19,lpt when you sign up for anything online put the websites name as your middle name that way when you receive spamadvert emails you will know who sold your info,https://www.reddit.com/r/LifeProTips/comments/5hexgl/lpt_when_you_sign_up_for_anything_online_put_the/,44034
363293,efb5dz,19,lpt add a sites name after your gmail address to track whos selling your data eg registering with johndoeamazongmailcom sends all email to johndoegmailcom this way when you get spam look at the to in the email it should have johndoeamazongmailcom revealing amazon sold your data,https://www.reddit.com/r/LifeProTips/comments/efb5dz/lpt_add_a_sites_name_after_your_gmail_address_to/,35282
385139,fzkztj,19,lpt if you get an email saying someone has digital evidence of you watching pornography or having sex and will distribute it to all your contacts unless you pay them bitcoin put that email right in your junk mail its a scam,https://www.reddit.com/r/LifeProTips/comments/fzkztj/lpt_if_you_get_an_email_saying_someone_has/,34087
467886,n4pnl7,19,lpt the next time youre waiting in a long line take that extra time to unsubscribe from junk email it can really help with workflow and will only take a few clicks to remove,https://www.reddit.com/r/LifeProTips/comments/n4pnl7/lpt_the_next_time_youre_waiting_in_a_long_line/,29342
475289,o4lwfe,19,lpt to declutter email type unsubscribe in your email search box and every email that comes from a promotion will come up then you can go through them and unsubscribe from them all,https://www.reddit.com/r/LifeProTips/comments/o4lwfe/lpt_to_declutter_email_type_unsubscribe_in_your/,28969
407500,hu5dw9,19,lpt when buying a house create a dedicated email address for the house and use it for everything but only for things related to the house mortgage insurance lawn person contractors etc its so much easier having it all in one place,https://www.reddit.com/r/LifeProTips/comments/hu5dw9/lpt_when_buying_a_house_create_a_dedicated_email/,19939
458679,lzpkjt,19,lpt dont click unsubscribe in an email that you didnt subscribe to in doing so you are only confirming that your email address is in use and ready to be targeted with more spam instead mark the email as spam blacklist in your email client,https://www.reddit.com/r/LifeProTips/comments/lzpkjt/lpt_dont_click_unsubscribe_in_an_email_that_you/,10447


None

Save topic as a string

In [160]:
def change_value(value):
    return f"Topic {int(np.round(value, 0))}"

# Apply the function to the column
data_top20_topics['topic'] = data_top20_topics['topic'].apply(change_value)

Save dataframe in new csv

In [161]:
data_top20_topics.drop(columns=["Unnamed: 0"], inplace=True)

In [162]:
data_top20_topics.to_csv("/content/drive/My Drive/UChicago/nlp/data_with_topics.csv")